<a href="https://colab.research.google.com/github/jerry-mouse-2333/colab_save/blob/master/train_raw_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ***step1. 导入必要的包***

In [48]:
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
import sys
import h5py
import tensorflow.compat.v1 as tf
from functools import partial
tf.reset_default_graph()
print('ready')

ready


## ***step2. 防 jupyter dead kernel***

In [49]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True

session = InteractiveSession(config=config)
session.close()

# InteractiveSession.close(session)
print('ready')

ready


## ***step3. make def for plot***

#### **step3.1. basic 画布和标签**

In [50]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

#定义子图、x轴、y轴的标签大小

(plt.rcParams['axes.labelsize'],plt.rcParams['xtick.labelsize'],plt.rcParams['ytick.labelsize'])  = (14,12,12)

#定义图像显示的dpi、保存的dpi，显示的窗口大小

(plt.rcParams['figure.dpi'],plt.rcParams['savefig.dpi'],plt.rcParams['figure.figsize'])=(300,300,(80,60))

print('ready')

ready


###### **step3.1.1. plot** (desired_shape:希望展示的图片大小)

In [51]:
def plot_image(image, desired_shape=[128, 128]):
    
    plt.imshow(image.reshape(desired_shape), cmap="Greys", interpolation="nearest")
    plt.axis("off")

def plot_multiple_images(images, n_rows, n_cols, pad=2):
    print(images)
    images = images - images.min()  # make the minimum == 0, so the padding looks white
    print(images)
    
    w,h = images.shape[1:]
    
    image = np.zeros(((w+pad)*n_rows+pad, (h+pad)*n_cols+pad))
    
    for y in range(n_rows):
        for x in range(n_cols):
            image[(y*(h+pad)+pad):(y*(h+pad)+pad+h),(x*(w+pad)+pad):(x*(w+pad)+pad+w)] = images[y*n_cols+x]
            
    plt.imshow(image, cmap="Greys", interpolation="nearest")
    plt.axis("off")

print('ready')

ready


#### **step3.2. where to save**

In [52]:
# Where to save the figures
root_dir = "/content/drive"
sub_dir = "/My Drive/result of trains"
path = os.path.join(root_dir,sub_dir,"images")
print(path)


/My Drive/result of trains/images


In [53]:

#tight_layout: 使子图布满图像区域
def save_fig(fig_id, tight_layout=True):
    save_path = os.path.join(path, fig_id + ".png")
    print("Saving figure", fig_id)
    os.makedirs(path, exist_ok=True)
    
#    if not os.path.exists(os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)):

#        os.makedirs(os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID))
    if tight_layout:
        plt.tight_layout()
    plt.savefig(save_name, format='png', dpi=300)
print('ready')

ready


## **step4. 散度**

In [54]:
def kl_divergence(p, q):
    # Kullback Leibler divergence
    return p * tf.log(p / q) + (1 - p) * tf.log((1 - p) / (1 - q))
print('ready')

ready


## **step4. 网络结构**
#### **step4.1. 参数设置**

In [55]:
filters=[32,16,32]
conv_kernel=3

batch_size = 64
ephochs = 10

#原始图片尺寸
width = 640
height = 480
depth = 3

#输入层
input_dim = (480,640)

#编码层输出
en_output=2

#解码层输出
output_dim = input_dim

#学习率与正则参数
learning_rate = 0.01
l2_reg = 0.0001

#散度参数
sparsity_target = 0.1
sparsity_weight = 0.2

#train_method2：全连接层
en1 = 300 #hid_1 of encoder
en2 = 150  
de1 = en2 #hid_1 of decoder
de2 = en1



print('ready')

ready


无奈之举，想设置kernel_size=5，strides=3，但无法使得输入输出一致

#### **step4.2. 网络结构**
###### **step4.21. encoder**

In [0]:
#收集变量绘制其histogram等
tensor_list=[]

with tf.name_scope("encoder") as scope:

    def en_module(en_inputs,en_output_dim,filters,k=conv_kernel):
        x=en_inputs
        i=0
        for _ in filters:
            print(_)
            # if (i==2):
            #   x = tf.layers.conv2d(x,_,3,2,name='conv_'+str(i))
            # else:
            x = tf.layers.conv2d(x,_,3,2,padding='same',name='conv_'+str(i))
            tensor_list.append(x)
            print(x.shape)
            tf.add_to_collection('reuse', x)
            x=tf.nn.leaky_relu(x, alpha=0.01,name='act_'+str(i))
            tensor_list.append(x)
            
            tf.add_to_collection('reuse', x)
            x = tf.layers.batch_normalization(x,axis=-1,name='batch_norm_'+str(i))
            tensor_list.append(x)
            
            tf.add_to_collection('reuse', x)
            i+=1
        
        shape=x.get_shape().as_list()

        print(shape)
        flat1 = tf.layers.flatten(x,name='flattened')
        #print(flat1.shape)

        #必须使用sigmoid激活使得KL_loss!=Nan
        half_output = my_dense_layer(flat1,en_output_dim,activation = "sigmoid",name="encoder_output")

        tf.add_to_collection('reuse', half_output)

        return shape,flat1,half_output


###### **step4.21. decoder**

In [0]:
with tf.name_scope("decoder") as scope:
  def de_module(de_inputs,depth,filters,k=conv_kernel):
    x = de_inputs
    i=0

    for _ in filters[::-1]:
      x = tf.layers.conv2d_transpose(x,_,3,2,padding='same',name='conv_tran_'+str(i+len(filters)))
      print(x.shape)
      x=tf.nn.leaky_relu(x, alpha=0.01,name='de_act_'+str(i+len(filters)))
      
      x = tf.layers.batch_normalization(x,axis=-1,name='de_batch_norm_'+str(i+len(filters)))
      i+=1
      

      #整合所有通道
    outputs_op = tf.layers.conv2d_transpose(x,depth,3,1,padding='same',activation="sigmoid",name='outputs_op')

      #观察最后的重建效果
    tf.add_to_collection('reuse', outputs_op)
    outputs=outputs_op
    return outputs



###### **step4.2.3. 正式布置graph**

In [58]:
tf.compat.v1.disable_eager_execution()
inputs = tf.placeholder(tf.float32, shape=[None,input_dim[0],input_dim[1],depth])#?,128,128
print(inputs)

Tensor("Placeholder:0", shape=(None, 480, 640, 3), dtype=float32)


In [59]:

#把input加入到将来要用的集合中
tf.add_to_collection('reuse', inputs)
print(inputs.shape)

#he_init = tf.contrib.layers.variance_scaling_initializer() # He initialization
he_init = tf.variance_scaling_initializer()
l2_regularizer = tf.keras.regularizers.l2(l2_reg)
my_dense_layer = partial(tf.layers.dense,activation=tf.nn.elu,
                             kernel_initializer=he_init,
                             kernel_regularizer=l2_regularizer)
# en_inputs = tf.expand_dims(inputs,axis=-1,name="encoder_input")
en_inputs=inputs

shape,flat1,half_output = en_module(en_inputs,en_output_dim=en_output,filters=filters)
print('shape_in_encoder:',shape)
print(half_output.name)


decode_input = my_dense_layer(half_output,np.prod(flat1.shape[1:]),name="decoder_input")#现为（？，32*32*8）
#print(decode_input)
decode_input_reshape= tf.reshape(decode_input,shape=(-1,shape[1],shape[2],shape[3]))#how to reshape it to(?,32,32,8)

outputs = de_module(decode_input_reshape,depth=depth,filters=filters)
print(outputs)
print(outputs.name)

encode_mean = tf.reduce_mean(half_output, axis=0)



(None, 480, 640, 3)
32
(None, 240, 320, 32)
16
(None, 120, 160, 16)
32
(None, 60, 80, 32)
[None, 60, 80, 32]
shape_in_encoder: [None, 60, 80, 32]
encoder_output/Sigmoid:0
(None, 120, 160, 32)
(None, 240, 320, 16)
(None, 480, 640, 32)
Tensor("outputs_op/Sigmoid:0", shape=(None, 480, 640, 3), dtype=float32)
outputs_op/Sigmoid:0


## step4 可视化设置

In [60]:
with tf.name_scope("loss") as scope:
    kl=kl_divergence(sparsity_target, encode_mean)

    sparsity_loss = tf.reduce_sum(kl_divergence(sparsity_target, encode_mean))
    reconstruction_loss = tf.reduce_mean(tf.square(outputs - en_inputs))
    loss = reconstruction_loss + sparsity_weight * sparsity_loss
    #tensor_list.append(loss)
optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss)

tf.summary.scalar("loss",loss)
# 用来显示标量信息，常用于画出loss、accuracy曲线
print(tensor_list)
for i in range(len(tensor_list)):
    name=tensor_list[i]
    #print(name.dtype)
    tf.summary.histogram(str(name),name) 
# 用来收集tensor，可显示出训练过程中变量的分布情况


init = tf.global_variables_initializer()
saver = tf.train.Saver() # not shown in the book
print('ok')

[<tf.Tensor 'conv_0/BiasAdd:0' shape=(None, 240, 320, 32) dtype=float32>, <tf.Tensor 'act_0:0' shape=(None, 240, 320, 32) dtype=float32>, <tf.Tensor 'batch_norm_0/FusedBatchNormV3:0' shape=(None, 240, 320, 32) dtype=float32>, <tf.Tensor 'conv_1/BiasAdd:0' shape=(None, 120, 160, 16) dtype=float32>, <tf.Tensor 'act_1:0' shape=(None, 120, 160, 16) dtype=float32>, <tf.Tensor 'batch_norm_1/FusedBatchNormV3:0' shape=(None, 120, 160, 16) dtype=float32>, <tf.Tensor 'conv_2/BiasAdd:0' shape=(None, 60, 80, 32) dtype=float32>, <tf.Tensor 'act_2:0' shape=(None, 60, 80, 32) dtype=float32>, <tf.Tensor 'batch_norm_2/FusedBatchNormV3:0' shape=(None, 60, 80, 32) dtype=float32>]
INFO:tensorflow:Summary name Tensor("conv_0/BiasAdd:0", shape=(None, 240, 320, 32), dtype=float32) is illegal; using Tensor__conv_0/BiasAdd_0___shape__None__240__320__32___dtype_float32_ instead.
INFO:tensorflow:Summary name Tensor("act_0:0", shape=(None, 240, 320, 32), dtype=float32) is illegal; using Tensor__act_0_0___shape__N

In [62]:
summary_save_dir = os.path.join(root_dir,sub_dir, "models")
os.makedirs(summary_save_dir, exist_ok=True)
print(summary_save_dir )
writer= tf.summary.FileWriter(summary_save_dir,tf.get_default_graph())
#获取当前默认的保存图，并在tensorboard graph中展示


merge_op = tf.summary.merge_all()
#执行所有summary操作
print("ready")

/My Drive/result of trains/models
ready


## **step5. 开始训练与结果保存**

In [64]:
data_path = '/content/drive/Shared drives/shared/dataset.h5'
h5_file=h5py.File(data_path,'r')
X_train=np.array(h5_file['X_train'])
X_test=np.array(h5_file['X_test'])

def get_batch(dataset):
    input_queue = tf.train.slice_input_producer([dataset],num_epochs=n_epochs,shuffle=True)
    X_batch = tf.train.batch(input_queue, batch_size=batch_size, num_threads=1, capacity=64)
    return X_batch
X_batch= get_batch(X_train) 

with tf.Session() as sess:
    init.run()
    sess.run(tf.local_variables_initializer())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess, coord)
    try:
        for epoch in range(epochs-8):
            n_batches = X_train.shape[0] // batch_size

            print(n_batches)
            for iteration in range(n_batches):
                #print("\r{}%".format(100 * iteration // n_batches), end="") # not shown in the book
                sys.stdout.flush()
                x_batch=sess.run(X_batch)
                #print(x_batch.shape)
                sess.run(training_op, feed_dict={inputs: x_batch})
                result=sess.run(merge_op, feed_dict={inputs: x_batch})
                writer.add_summary(result,iteration)
                
            reconstruction_loss_val, sparsity_loss_val, loss_val = sess.run([reconstruction_loss,sparsity_loss, loss], feed_dict={inputs: x_batch})
            #print(sess.run(kl,feed_dict={X: x_batch}))
            print("\r{}".format(epoch), "Train MSE:", reconstruction_loss_val, "\tSparsity loss:",sparsity_loss_val, "\tTotal loss:", loss_val)           # not shown
              
    except tf.errors.OutOfRangeError:
        print("done")
    finally:
        coord.request_stop()
    coord.join(threads)
    
    save_dir = '/content/drive/My Drive/result of trains'
    save_path = os.path.join(save.dir,"my_model_all_layers.ckpt")

    saver.save(sess, save_path) 
    print('ok')


KeyboardInterrupt: ignored